# Normalizar e padronizar variáveis

In [1]:
from joblib import dump, load
import pandas as pd

In [2]:
caminho = '../../util/data/'

## seleção das variáveis numéricas

In [3]:
# carregamento dos dataframes e seleção das variaveis numericas

categorias = load(caminho+'df_categorias.z')
categorias_numerica= categorias.select_dtypes(include=['int64', 'float64']) # seleciona apenas as variaveis numericas do tipo int64 e float64

clientes = load(caminho+'df_clientes.z')
clientes_numerica= clientes.select_dtypes(include=['int64', 'float64'])

produtos = load(caminho+'df_produtos.z')
produtos_numerica = produtos.select_dtypes(include=['int64', 'float64'])

# exibe informações das variaveis numericas de cada dataframe
print('[CATEGORIAS]\n',categorias_numerica.describe())
print('[CLIENTES]\n',clientes_numerica.describe())
print('[PRODUTOS]\n',produtos_numerica.describe())

[CATEGORIAS]
            codigo
count   74.000000
mean   303.851351
std    155.563288
min      2.000000
25%    343.000000
50%    381.500000
75%    399.750000
max    419.000000
[CLIENTES]
             codigo     vendedor
count  3132.000000  3132.000000
mean   6786.610153    67.129630
std    2827.491552    30.297795
min       1.000000     7.000000
25%    5194.750000    54.000000
50%    7997.500000    82.000000
75%    8844.250000    89.000000
max    9730.000000   101.000000
[PRODUTOS]
        codigo_categoria   preco_venda
count       9589.000000   9592.000000
mean          81.104390    304.013080
std          130.790131   1366.995745
min            2.000000      0.000000
25%            3.000000     19.735000
50%            8.000000     65.540000
75%           64.000000    214.000000
max          368.000000  51283.640000


## `df_produtos`

### → variáveis numéricas

com o output, pode-se avaliar:

`categorias`: somente possui uma variável numérica para servir de id da categoria, será descartada

`clientes`: só possui variáveis que servem como identificadores **('codigo','vendedor')**, serão descartados

`produtos`: também possui variáveis identificadoras **('codigo_categoria)** que será descartada nos procedimentos. será necessário aplicar uma normalização em 'preco_venda' por apresentar valores muito distantes  


In [4]:
produtos_numerica = produtos_numerica.drop(columns=['codigo_categoria']) # descartando a coluna id para a padronização

produtos_numerica.head(10)

,preco_venda
0,0.00
1,8.93
2,0.00
3,8.38
4,67.91
5,441.54
6,14.74
7,26.52
8,954.28
9,16.00


In [ ]:
 # importando a bibliotecas necessárias
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [6]:
scaler = StandardScaler() # declarando o scaler

scaler.fit(produtos_numerica) # aplicando no produtos
produtos_numerica_std = scaler.transform(produtos_numerica)
produtos_numerica_std = pd.DataFrame(produtos_numerica_std, columns=produtos_numerica.columns) # coluna padronizada

In [7]:
produtos_numerica_std.head(10)

,preco_venda
0,-0.222407
1,-0.215874
2,-0.222407
3,-0.216276
4,-0.172726
5,0.100610
6,-0.211623
7,-0.203005
8,0.475715
9,-0.210702


### → variáveis categóricas

In [ ]:
produtos = produtos.drop(columns=['codigo_barras']) #removendo a coluna codigo_barras por ser nula

In [15]:
# seleciona somente as variaveis categoricas de produtos
produtos_categoricas = produtos.select_dtypes(include=['object', 'category'])

In [18]:
produtos_categoricas.head(10)

,codigo,descricao,unidade,marca,categoria,classificacao
0,194408,CANETA REF 35629,UN,BIC,CANETA,MERCADORIA PARA REVENDA
1,194412,CANETA REF 35633,UN,BIC,CANETA,MERCADORIA PARA REVENDA
2,194413,CANETA REF 35634,UN,BIC,CANETA,MERCADORIA PARA REVENDA
3,006467,CANETA REF 5968,UN,BIC,CANETA,MERCADORIA PARA REVENDA
4,004559,CANETA REF 4092,UN,BIC,CANETA,MERCADORIA PARA REVENDA
5,006654,ENVELOPE REF 6152,UN,3M,ENVELOPE,MERCADORIA PARA REVENDA
6,005543,CANETA REF 5066,UN,BIC,CANETA,MERCADORIA PARA REVENDA
7,005422,CANETA REF 4947,UN,BIC,CANETA,MERCADORIA PARA REVENDA
8,193691,TINTA REF 34938,UN,FABER CASTELL,TINTA,MERCADORIA PARA REVENDA
9,161291,CANETA REF 16450,UN,BIC,CANETA,MERCADORIA PARA REVENDA


com a visualização, podemos verificar que as colunas:

`codigo`: é um identificador, não vai trazer nenhuma informação, será descartada

`unidade`: possui somente dois tipos, UN e CJ, será aplicado a técnica **one-hot encoding**

`descricao`: não acrescenta em nada aparentemente, será descartada

`marca`, `categoria`, `classificacao`: possuem muitos tipos, será aplicada a técnica **label encoding**

In [ ]:
# remoção da coluna codigo e descricao
produtos_categoricas = produtos_categoricas.drop(columns=['codigo']) 
produtos_categoricas = produtos_categoricas.drop(columns=['descricao']) 

In [ ]:
df = pd.DataFrame({'unidade': ['UN', 'CJ']})
df_onehot = pd.get_dummies(df, columns=['unidade'])
print(df_onehot)

In [ ]:
le = LabelEncoder()
produtos_categoricas[''] = le.fit_transform(produtos_categoricas['nome_fantasia'])

In [ ]:
dump(produtos_numerica_std, caminho+'produtos_std.z')

# `df_unidades`

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [6]:
df_unidades = pd.read_csv("../../util/data/unidades.csv")

In [7]:
# Codificando a coluna 'status' (sim/não) em 1 e 0
df_unidades['status_cod'] = df_unidades['status'].map({'sim': 1, 'não': 0})

In [8]:
# Codificando a coluna 'nome_fantasia' com Label Encoding
le = LabelEncoder()
df_unidades['nome_fantasia_cod'] = le.fit_transform(df_unidades['nome_fantasia'])

In [9]:
print(df_unidades.head())

   codigo_unidade    nome_fantasia status  status_cod  nome_fantasia_cod
0           10003   L03 - SALVADOR    sim           1                  2
1           10002     L02 - SOBRAL    sim           1                  1
2           10001  L01 - FORTALEZA    sim           1                  0


# `df_vendedores`

In [10]:
from joblib import load
df_vendedores = load("../../util/data/df_vendedores.z")

In [11]:
print(df_vendedores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   codigo         13 non-null     int64 
 1   nome_completo  13 non-null     object
 2   nome_reduzido  13 non-null     object
dtypes: int64(1), object(2)
memory usage: 444.0+ bytes
None


In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [13]:
data = {
    "codigo": [11, 51, 88, 89, 54, 56, 94, 79, 58, 9, 90, 32, 101],
    "nome_completo": [
        "ADALBERTO MONTENEGRO SILVA", "DIOGO MONTEIRO JUNIOR", "PAULO GIRÃO MONTE",
        "ALEXANDRA SAMPAIO OLIVEIRA", "PATRICIA LEITE NOBRE", "ALBERTO DAMASCENO NOBREGA",
        "MARIO LOPES NASCIMENTO", "VITOR OLIVEIRA SANTOS", "CLAUDIA MARIA DAMASCENO",
        "MARCIO SANTOS DE NOBREGA", "ULISSES SANTOS DE OLIVEIRA", "MARIA DAS DORES SILVA",
        "VENDEDOR MOBILE"
    ],
    "nome_reduzido": [
        "ADALBERTO", "DIOGO", "PAULO MONTE", "ALEXANDRA", "PATRICIA", "ALBERTO",
        "MARIO LOPES", "VITOR", "CLAUDIA", "MARCIO SANTOS", "ULISSES", "MARIA DAS DORES", "VENDEDOR MOBILE"
    ]
}

In [14]:
df_vendedores = pd.DataFrame(data)

In [15]:
# Aplicando LabelEncoder para nome_completo e nome_reduzido
le_completo = LabelEncoder()
le_reduzido = LabelEncoder()

df_vendedores['nome_completo_cod'] = le_completo.fit_transform(df_vendedores['nome_completo'])
df_vendedores['nome_reduzido_cod'] = le_reduzido.fit_transform(df_vendedores['nome_reduzido'])

In [16]:
print(df_vendedores)

    codigo               nome_completo    nome_reduzido  nome_completo_cod  \
0       11  ADALBERTO MONTENEGRO SILVA        ADALBERTO                  0   
1       51       DIOGO MONTEIRO JUNIOR            DIOGO                  4   
2       88           PAULO GIRÃO MONTE      PAULO MONTE                  9   
3       89  ALEXANDRA SAMPAIO OLIVEIRA        ALEXANDRA                  2   
4       54        PATRICIA LEITE NOBRE         PATRICIA                  8   
5       56   ALBERTO DAMASCENO NOBREGA          ALBERTO                  1   
6       94      MARIO LOPES NASCIMENTO      MARIO LOPES                  7   
7       79       VITOR OLIVEIRA SANTOS            VITOR                 12   
8       58     CLAUDIA MARIA DAMASCENO          CLAUDIA                  3   
9        9    MARCIO SANTOS DE NOBREGA    MARCIO SANTOS                  5   
10      90  ULISSES SANTOS DE OLIVEIRA          ULISSES                 10   
11      32       MARIA DAS DORES SILVA  MARIA DAS DORES         

In [ ]:
# obs.: junta com o original e salva em um só
dump(vendedores_std, caminho+'vendedores_std.z')

# `df_vendas`

In [17]:
from joblib import load
df_vendas = load("../../util/data/df_vendas.z")

In [18]:
print(df_vendas.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10418 entries, 0 to 10427
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   codigo_empresa    10418 non-null  int64         
 1   tipo_pedido       10418 non-null  object        
 2   centro_resultado  10418 non-null  int64         
 3   documento         10418 non-null  int64         
 4   codigo_cliente    10418 non-null  int64         
 5   codigo_empregado  10418 non-null  int64         
 6   data_emissao      10418 non-null  datetime64[ns]
 7   hora_emissao      10418 non-null  object        
 8   data_fechamento   10418 non-null  datetime64[ns]
 9   hora_fechamento   10418 non-null  object        
 10  data_faturamento  10418 non-null  datetime64[ns]
 11  codigo_produto    10418 non-null  int64         
 12  quantidade        10418 non-null  float64       
 13  total             10418 non-null  float64       
 14  custo_medio       10418 non

In [ ]:
df_vendas.head(5)

,codigo_empresa,tipo_pedido,centro_resultado,documento,codigo_cliente,codigo_empregado,data_emissao,hora_emissao,data_fechamento,hora_fechamento,data_faturamento,codigo_produto,quantidade,total,custo_medio,valor_desconto
0,2,PV,2,24560,4395,56,2023-03-21,12:10:07,2023-03-21,12:12:40,2023-03-21,194959,8.0,64.0,5.2870,20.64
1,1,PV,2,22433,236,9,2023-02-13,10:40:44,2023-04-27,07:18:40,2023-04-27,195052,6.0,75.0,5.2931,0.00
2,2,PV,2,24560,4395,56,2023-03-21,12:10:07,2023-03-21,12:12:40,2023-03-21,164,3.0,75.0,15.3550,16.80
3,2,PV,2,24560,4395,56,2023-03-21,12:10:07,2023-03-21,12:12:40,2023-03-21,201,3.0,144.0,31.3017,54.72
4,2,PV,2,24560,4395,56,2023-03-21,12:10:07,2023-03-21,12:12:40,2023-03-21,191696,2.0,60.0,15.3650,11.84
5,2,PV,2,24560,4395,56,2023-03-21,12:10:07,2023-03-21,12:12:40,2023-03-21,186187,3.0,231.0,53.2162,94.68
6,1,PV,2,22433,236,9,2023-02-13,10:40:44,2023-04-27,07:18:40,2023-04-27,7242,2.0,30.0,3.8400,0.00
7,1,PV,2,22433,236,9,2023-02-13,10:40:44,2023-04-27,07:18:40,2023-04-27,195996,2.0,130.0,41.0500,0.00
8,1,PV,2,22433,236,9,2023-02-13,10:40:44,2023-04-27,07:18:40,2023-04-27,56,6.0,228.0,11.6863,0.00
9,1,PV,2,22433,236,9,2023-02-13,10:40:44,2023-04-27,07:18:40,2023-04-27,7207,2.0,44.0,7.1500,0.00


In [20]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [21]:
# Selecionando as colunas numéricas relevantes
num_cols = ['quantidade', 'total', 'custo_medio', 'valor_desconto']

In [22]:
# Inicializando os scalers
minmax = MinMaxScaler()
zscore = StandardScaler()

In [23]:
# Aplicando Min-Max Normalization
df_minmax = pd.DataFrame(
    minmax.fit_transform(df_vendas[num_cols]),
    columns=[col + '_minmax' for col in num_cols],
    index=df_vendas.index
)

In [24]:
# Aplicando Z-score Standardization
df_zscore = pd.DataFrame(
    zscore.fit_transform(df_vendas[num_cols]),
    columns=[col + '_zscore' for col in num_cols],
    index=df_vendas.index
)

In [25]:
df_vendas = pd.concat([df_vendas, df_minmax, df_zscore], axis=1)

In [26]:
# One-hot encoding para tipo_pedido
df_vendas = pd.get_dummies(df_vendas, columns=['tipo_pedido'], prefix='pedido')

In [27]:
print(df_vendas.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10418 entries, 0 to 10427
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   codigo_empresa         10418 non-null  int64         
 1   centro_resultado       10418 non-null  int64         
 2   documento              10418 non-null  int64         
 3   codigo_cliente         10418 non-null  int64         
 4   codigo_empregado       10418 non-null  int64         
 5   data_emissao           10418 non-null  datetime64[ns]
 6   hora_emissao           10418 non-null  object        
 7   data_fechamento        10418 non-null  datetime64[ns]
 8   hora_fechamento        10418 non-null  object        
 9   data_faturamento       10418 non-null  datetime64[ns]
 10  codigo_produto         10418 non-null  int64         
 11  quantidade             10418 non-null  float64       
 12  total                  10418 non-null  float64       
 13  custo_

In [28]:
print(df_vendas.describe())

       codigo_empresa  centro_resultado     documento  codigo_cliente  \
count    10418.000000      10418.000000  10418.000000    10418.000000   
mean         1.583797          4.140910  22114.464293     5790.050490   
min          1.000000          2.000000  18228.000000        2.000000   
25%          1.000000          2.000000  20850.250000     4798.000000   
50%          2.000000          2.000000  22135.000000     6863.000000   
75%          2.000000          7.000000  23375.750000     6863.000000   
max          3.000000         14.000000  24601.000000     9708.000000   
std          0.633558          2.881906   1425.949460     2558.593989   

       codigo_empregado                   data_emissao  \
count      10418.000000                          10418   
mean          63.667691  2023-02-08 16:44:44.277212416   
min            8.000000            2022-12-06 00:00:00   
25%           51.000000            2023-01-19 00:00:00   
50%           63.000000            2023-02-08 00:00:

In [ ]:
dump(df_vendas, caminho+'vendas_std.z') 